In [ ]:
# !pip install uv
# !uv pip install git+https://github.com/ysenarath/AIT-724-HandsOn-2.git
# !uv pip install --upgrade "numpy==1.26.4" "scipy==1.12.0"

In [ ]:
import copy  # for deep copying objects
import gzip  # for handling gzip compressed files
import os  # for operating system interactions
import random  # for random number generation / sampling
import shutil  # for high-level file operations
import zipfile  # for handling zip files
from datetime import timedelta  # for time duration representation
from pathlib import Path  # for filesystem path manipulations
from typing import Any, Callable, cast  # for type hinting

from community import community_louvain  # for community detection
import holoviews as hv  # for interactive visualizations
import hvplot.networkx as hvnx  # for network visualizations
import igraph as ig  # for faster graph analysis
import matplotlib.pyplot as plt  # for static visualizations
import networkx as nx  # for graph creation and manipulation
import numpy as np  # for numerical operations
import pandas as pd  # for data manipulation
import requests  # for downloading data
import seaborn as sns  # for statistical data visualization
from networkx.drawing.layout import forceatlas2_layout  # for ForceAtlas2 layout
from networkx.drawing.nx_pydot import graphviz_layout  # for Graphviz layouts
from tqdm import auto as tqdm  # for progress bars

forceatlas2_layout = cast(
    Callable[..., Any], forceatlas2_layout
)  # type hinting workaround (you can ignore this line)

hv.extension("bokeh")  # initialize Holoviews with Bokeh backend

In [ ]:
def display_network(
    G: nx.Graph,
    edge_sample_size: int | None = None,
    date_format: str = "%Y-%m-%d %H:%M:%S",
    layout: str = "forceatlas2",
    layout_kwargs: dict[str, Any] | None = None,
    node_size: int = 10,
    arrowhead_length: float = 0.01,
):
    # Utility function to visualize a NetworkX graph using hvplot
    # Sample graph to manageable size
    if edge_sample_size is not None and len(G.edges) > edge_sample_size:
        # nodes_sample = random.sample(list(G.nodes()), sample_size)
        # T = copy.deepcopy(G.subgraph(nodes_sample))
        # sample edges and then select the subgraph induced by those edges
        edges_sample = random.sample(list(G.edges()), edge_sample_size)
        T = copy.deepcopy(G.edge_subgraph(edges_sample))
    else:
        T = copy.deepcopy(G)

    # Convert Timestamp attributes to strings
    for n, data in T.nodes(data=True):
        for k, v in list(data.items()):
            if isinstance(v, pd.Timestamp):
                data[k] = v.strftime(date_format)

    for u, v, data in T.edges(data=True):
        for k, val in list(data.items()):
            if isinstance(val, pd.Timestamp):
                data[k] = val.strftime(date_format)

    if layout == "forceatlas2":
        default_kwargs = {
            "max_iter": 100,
            "jitter_tolerance": 1.0,
            "scaling_ratio": 2.0,
            "gravity": 1.0,
        }
        default_kwargs.update(layout_kwargs or {})
        pos = forceatlas2_layout(T, **default_kwargs)
    else:
        pos = graphviz_layout(T, prog=layout or "neato")

    # Create hvplot
    plot = hvnx.draw(
        T,
        pos,
        node_size=node_size,
        node_color="lightblue",
        edge_color="gray",
        with_labels=True,
        # arrow head size = 0.02,
        arrows=True,
        arrowhead_length=arrowhead_length,
    ).opts(
        width=900,
        height=700,
        tools=["hover", "box_zoom", "wheel_zoom", "save", "reset"],
        title=f"Interactive Graph (|N|={len(T)}, |E|={len(T.edges)})",
    )

    # Add hover info dynamically
    hover_data = []
    for n, d in T.nodes(data=True):
        hover_data.append({"id": n, **d})
        # You can attach hover info via node attributes if needed

    return plot

# Download the Files

- The files are available at https://snap.stanford.edu/data/higgs-twitter.html

- Base URL: https://snap.stanford.edu/data/higgs-*

| File Name                   | Description                                                                                    |
| --------------------------- | ---------------------------------------------------------------------------------------------- |
| social_network.edgelist.gz  | Friends/follower graph (directed)                                                              |
| retweet_network.edgelist.gz | Graph of who retweets whom (directed and weighted)                                             |
| reply_network.edgelist.gz   | Graph of who replies to who (directed and weighted)                                            |
| mention_network.edgelist.gz | Graph of who mentions whom (directed and weighted)                                             |
| higgs-activity_time.txt.gz  | The dataset provides information about activity on Twitter during the discovery of Higgs boson |


In [ ]:
def download_file(url, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    local_filename = os.path.join(dest_folder, url.split("/")[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get("content-length", 0))
        with (
            open(local_filename, "wb") as f,
            tqdm.tqdm(
                desc=local_filename,
                total=total_size,
                unit="iB",
                unit_scale=True,
                unit_divisor=1024,
            ) as bar,
        ):
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
                bar.update(len(chunk))
    return local_filename


def unzip_file(zip_path, extract_to):
    if zipfile.is_zipfile(zip_path):
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(extract_to)
    elif zip_path.endswith(".gz"):
        with gzip.open(zip_path, "rb") as f_in:
            with open(os.path.splitext(zip_path)[0], "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)


def main():
    base_url = "https://snap.stanford.edu/data/higgs-"
    files = [
        "social_network.edgelist.gz",
        "retweet_network.edgelist.gz",
        "reply_network.edgelist.gz",
        "mention_network.edgelist.gz",
        "activity_time.txt.gz",
    ]
    dest_folder = "higgs_twitter_data"

    for file_name in files:
        url = base_url + file_name
        output_path = Path(dest_folder) / ("higgs-" + file_name.replace(".gz", ""))
        if output_path.exists():
            print(f"{file_name} already exists. Skipping download.")
            continue
        print(f"Downloading {url}...")
        downloaded_file = download_file(url, dest_folder)
        print(f"Unzipping {downloaded_file}...")
        unzip_file(downloaded_file, dest_folder)
        os.remove(downloaded_file)  # Clean up the .gz file
        print(f"Finished processing {file_name}.\n")


main()

# Understanding Edge List File Format

- For this exercise, we will only explore the the activity time file: `higgs-activity_time.txt.gz`
- This file contains four columns: userA userB timestamp interaction-type
- Every edge is directed from userA to userB (e.g., userA replies to userB, this means information flows from userB to userA)
- First let's try to load this file using pandas and explore its contents.


In [ ]:
TIMESTAMP = "timestamp"

df = pd.read_csv(
    os.path.join("higgs_twitter_data", "higgs-activity_time.txt"),
    sep=" ",
    header=None,
    names=["source", "target", TIMESTAMP, "type"],
)

print(f"Total interactions in dataset: {len(df)}")

# Convert timestamp to datetime object
df[TIMESTAMP] = pd.to_datetime(df[TIMESTAMP], unit="s")

plt.figure(figsize=(8, 5))
sns.histplot(data=df, x="type", discrete=True)
plt.title("Distribution of Interaction Types")
plt.xlabel("Interaction Type")
plt.ylabel("Count")
plt.show()

In [ ]:
df = df[df["type"] == "RE"]  # for the assignment this has to be changed
start_date = df[TIMESTAMP].min()

# If one is interested in building a network of how information flows,
#   then the direction of RT should be reversed when used in the analysis.
#   ref. https://snap.stanford.edu/data/higgs-twitter.html
df = df.rename(columns={"source": "target", "target": "source"})

total_number_of_nodes = df[["source", "target"]].stack().nunique()
total_number_of_edges = len(df)

print(f"Total number of unique nodes: {total_number_of_nodes}")
print(f"Total number of edges: {total_number_of_edges}")

# let's look at the time range of the interactions
min_time = df[TIMESTAMP].min()
max_time = df[TIMESTAMP].max()
print(f"Time range of interactions: {min_time} to {max_time}")

df.head(3)

In [ ]:
G: nx.DiGraph = nx.from_pandas_edgelist(
    df,
    source="source",
    target="target",
    edge_attr=[
        TIMESTAMP,
        "type",  # type will always be "RE" here (or whatever we filtered for)
    ],
    create_using=nx.DiGraph,
)

print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

In [ ]:
display_network(G, edge_sample_size=100)

In [ ]:
# Visualise diffusion: e.g., plot number of new retweeters over time.
temp_df = pd.DataFrame(
    [
        {
            TIMESTAMP: data[TIMESTAMP],
            "type": data["type"],
        }
        for u, v, data in G.edges(data=True)
    ]
)
plt.figure(figsize=(12, 6))
sns.histplot(
    temp_df,
    x=TIMESTAMP,
    bins=50,
    kde=False,
    hue="type",
    multiple="stack",
)
plt.show()

In [ ]:
# Compute degree
print("Computing node degrees...")

degree_dict = dict(G.degree())
nx.set_node_attributes(G, degree_dict, "degree")

# in degree
in_degree_dict = dict(G.in_degree())
nx.set_node_attributes(G, in_degree_dict, "in_degree")

In [ ]:
# Compute betweenness centrality

# # print("Computing betweenness centrality...")
# # takes around 15 mins-45mins
# centrality_dict = nx.betweenness_centrality(G)
# nx.set_node_attributes(G, centrality_dict, "betweenness_centrality")

# follwing is a more efficient way using igraph, commented out for now

# compute betweenness centrality for all vertices using igraph
print("Computing betweenness centrality using igraph...")
G_ig = ig.Graph.from_networkx(G)
vertex_betweenness = G_ig.betweenness()

# put it back to networkx graph
# betweenness_dict = {v.index: float(vertex_betweenness[v.index]) for v in G_ig.vs}
names = [v["_nx_name"] for v in G_ig.vs]
betweenness_dict = dict(zip(names, vertex_betweenness))
nx.set_node_attributes(G, betweenness_dict, "betweenness_centrality")

In [ ]:
# closeness centrality
print("Computing closeness centrality...")

closeness_dict = nx.closeness_centrality(G)

nx.set_node_attributes(G, closeness_dict, "closeness_centrality")

In [ ]:
# Run PageRank algorithm
print("Computing PageRank...")

pagerank_dict = nx.pagerank(G, alpha=0.85)

nx.set_node_attributes(G, pagerank_dict, "pagerank")

In [ ]:
# Compute clustering coefficient
print("Computing clustering coefficient...")

clustering_dict = nx.clustering(G.to_undirected())

nx.set_node_attributes(G, clustering_dict, "clustering_coefficient")

In [ ]:
# Add modularity class using the Louvain method
partition = community_louvain.best_partition(G.to_undirected())

for i, (key, value) in enumerate(partition.items()):
    # check the random 5 nodes
    print(f"Node {key}: Modularity Class {value}")
    if i >= 4:
        break

nx.set_node_attributes(G, partition, "modularity_class")

# print the number of nodes in each modularity class
modularity_classes = pd.Series(partition).value_counts(sort=True, ascending=False)

# top 5 largest classes
print("Top 5 largest modularity classes:")
print(modularity_classes.head(5))

In [ ]:
temp_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient="index")

# Follwing line can be used to see the distribution of modularity classes
# temp_df["modularity_class"].value_counts().head(5)

In [ ]:
# Common measure for quantifying influence of bloggers is to use in-degree centrality
# - This is the number of users who follow a person on Twitter.
# In-links are sparse
# - More detailed analysis is required to measure influence

in_degrees: dict[str, float] = nx.get_node_attributes(G, "in_degree")
threshold = np.percentile(list(in_degrees.values()), 99)  # top 1%

influencers = [n for n, deg in in_degrees.items() if deg >= threshold]
regulars = [n for n in G.nodes if n not in influencers]

print(f"Influencers: {len(influencers)} | Regulars: {len(regulars)}")

# add this back to the network G
influencer_dict = {n: (n in influencers) for n in G.nodes}
nx.set_node_attributes(G, influencer_dict, "is_influencer")

In [ ]:
# show node & edge attributes
print("Node attribute names for first node:")
first_node = list(G.nodes)[0]
print(G.nodes[first_node].keys())

print("Edges attribute names for first node:")
first_edge = list(G.edges)[0]
print(G.edges[first_edge].keys())

In [ ]:
def group_timestamps(
    G: nx.DiGraph,
    node: Any,
    time_window: timedelta | None,
    timestamp_col: str = "timestamp",
) -> list[list[pd.Timestamp]]:
    """
    We can assume that the interactions are about the same post if it happens within time_window.
    We can use this function to group timestamps of edges (intractions) from a node (user)
    into clusters based on time_window.

    Example:
    |       |   |      |                |       |
    t1      t2  t3     t4               t5      t6

    i0: [t1], [t2], [t3], [t4], [t5], [t6]

    t2 - t1 < time_window, therefore merge
    i0: [t1, t2], [t3], [t4], [t5], [t6]

    t3 - t2 < time_window, therefore merge
    i0: [t1, t2, t3], [t4], [t5], [t6]

    t4 - t3 < time_window, therefore merge
    i0: [t1, t2, t3, t4], [t5], [t6]

    t5 - t4 >= time_window, therefore do not merge
    i1: [t1, t2, t3, t4], [t5, t6]
    """
    # Group timestamps of edges from a node into clusters based on time_window
    # - first, get all timestamps of edges from the node
    items = []
    for source, target, data in G.edges(node, data=True):
        dt = data[timestamp_col]
        items.append(dt)
    # - sort timestamps to ensure chronological order
    items = sorted(items)
    if time_window is None:
        # - if no time window is specified, return all timestamps as a single group
        groups = [items]
    else:
        # - cluster timestamps based on time_window
        groups = [[x] for x in items]
        i = 0
        while i < len(groups) - 1:
            # - if the time difference between two consecutive timestamps is less than time_window, merge them
            # -- groups[i][-1] is the last timestamp in the current group
            # -- groups[i + 1][0] is the first timestamp in the next group
            if (groups[i + 1][0] - groups[i][-1]) < time_window:
                groups[i].extend(groups[i + 1])
                # - remove the next group after merging
                # -- when groups[i + 1] is deleted, groups[i + 2] becomes the new groups[i + 1]
                del groups[i + 1]
                # - do not increment i, as we need to check the merged group with the new next one
            else:
                i += 1
    return groups


def build_cascade_from_root(
    G: nx.DiGraph,
    root: Any,
    timestamp_cluster: list[pd.Timestamp],
    visited: set,
    time_window: timedelta | None = None,
    timestamp_col: str = "timestamp",
) -> nx.DiGraph:
    # Start a cascade
    cascade_nodes = set([root])
    cascade_edges = dict()
    queue = [(root, ts) for ts in timestamp_cluster]
    visited.add(root)
    while queue:
        current, t_curr = queue.pop(0)
        neighbors = sorted(
            G.neighbors(current),
            key=lambda nbr: G.get_edge_data(current, nbr, default={})[timestamp_col],
        )
        for nbr in neighbors:
            edge = G.get_edge_data(current, nbr, default={})
            # typeof datetime is pandas.Timestamp
            t_edge = edge[timestamp_col]
            # convert to real time and print
            if time_window is not None and (t_edge - t_curr > time_window):
                continue
            if nbr not in visited:
                cascade_nodes.add(nbr)
                cascade_edges[(current, nbr)] = edge
                queue.append((nbr, t_edge))
                visited.add(nbr)
    cascade = nx.DiGraph()
    cascade.add_nodes_from(cascade_nodes)
    cascade_edges_with_data = [
        (src, tgt, data) for (src, tgt), data in cascade_edges.items()
    ]
    cascade.add_edges_from(cascade_edges_with_data)
    return cascade


def build_cascades(
    G: nx.DiGraph,
    time_window: timedelta | None = None,
    min_cascade_size: int = 1,
    timestamp_col: str = "timestamp",
) -> list[nx.DiGraph]:
    cascades = []
    visited = set()
    for root in G.nodes():
        if root in visited:
            continue
        for timestamp_cluster in group_timestamps(
            G, root, time_window=time_window, timestamp_col=timestamp_col
        ):
            cascade = build_cascade_from_root(
                G,
                root,
                timestamp_cluster=timestamp_cluster,
                visited=visited,
                time_window=time_window,
                timestamp_col=timestamp_col,
            )
            cascades.append(cascade)
    # filter cascades with size > 1
    cascades = [c for c in cascades if len(c) > min_cascade_size]
    return cascades

In [ ]:
cascades = build_cascades(G, time_window=None, min_cascade_size=10)
print(f"Total cascades found: {len(cascades)}")

largest_cascade = max(cascades, key=len)
print(f"Largest cascade size: {len(largest_cascade)}")

# size of the cascades
cascade_sizes = [len(c) for c in cascades]
plt.figure(figsize=(10, 6))
sns.histplot(cascade_sizes, bins=30, kde=False)
plt.title("Cascade Size Distribution")
plt.xlabel("Cascade Size")
plt.ylabel("Frequency")
plt.show()

In [ ]:
cascades = build_cascades(G, time_window=timedelta(minutes=1), min_cascade_size=10)
print(f"Total cascades found: {len(cascades)}")

largest_cascade = max(cascades, key=len)
print(f"Largest cascade size: {len(largest_cascade)}")

# size of the cascades
cascade_sizes = [len(c) for c in cascades]
plt.figure(figsize=(10, 6))
sns.histplot(cascade_sizes, bins=30, kde=False)
plt.title("Cascade Size Distribution")
plt.xlabel("Cascade Size")
plt.ylabel("Frequency")
plt.show()

In [ ]:
display_network(largest_cascade, layout="twopi")

In [ ]:
# Get Node data about node 677
node_id = 677
print(f"Node {node_id} data:")
print(G.nodes[node_id])

In [ ]:
# compare the cascade sizes of influencers vs regular users
# You may have to change the time_window parameter for the assignment.
cascades = build_cascades(G, time_window=None, min_cascade_size=1)


def get_roots(G: nx.DiGraph) -> list[Any]:
    # roots are nodes with in-degree 0
    roots = [n for n in G.nodes() if G.in_degree(n) == 0]
    return roots


for cascade in cascades:
    roots = get_roots(cascade)
    if not roots:
        continue
    # get cascade size
    cascade_size = len(cascade)
    for root in roots:
        if "cascade_size" in G.nodes[root]:
            G.nodes[root]["cascade_size"] = max(
                G.nodes[root]["cascade_size"], cascade_size
            )
        else:
            G.nodes[root]["cascade_size"] = cascade_size

In [ ]:
temp_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient="index")
temp_df = temp_df.dropna(subset=["cascade_size"])

temp_df["is_influencer_yn"] = temp_df["is_influencer"].apply(
    lambda x: "Influencer" if x else "Regular"
)

sns.boxplot(
    data=temp_df,
    x="is_influencer_yn",
    y="cascade_size",
)
plt.yscale("log")
plt.title("Cascade Size by Influencer Status")
plt.xlabel("User Type")
plt.ylabel("Cascade Size (log scale)")
plt.show()